In [271]:
import pandas as pd
import numpy as np

from random import sample
from scipy import stats

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import seaborn as sns

In [298]:
file = 'GM23_Master Sales Table (FINAL).xlsx'

In [299]:
base_data = (pd.read_excel(file)
             .replace('no data', np.nan)
             [['Year 1 Sales - Inf', 'Year 2 Sales - Inf', 
               'Year 3 Sales - Inf', 'Year 4 Sales - Inf', 
               'Year 5 Sales - Inf']]
             .dropna(axis=0, how='any')
             .reset_index()
             .drop('index', axis=1))

# add 1 to every value and take log
base_data = (base_data+1).transform(lambda x : np.log(x))

In [274]:
base_data

,Year 1 Sales - Inf,Year 2 Sales - Inf,Year 3 Sales - Inf,Year 4 Sales - Inf,Year 5 Sales - Inf
0,0.458040,2.974573,4.007656,4.764603,5.298983
1,5.495585,5.919769,6.343687,6.375090,6.416497
2,3.112891,4.764430,5.807421,6.493077,6.983779
3,3.051184,6.080072,7.426026,7.856537,8.160062
4,5.402336,6.544056,6.790240,7.095385,7.359181
...,...,...,...,...,...
186,7.230494,8.598712,9.013016,9.139398,9.248695
187,1.503272,4.028479,5.011051,5.382642,5.596714
188,4.444369,5.244573,5.763279,5.993623,6.060906
189,3.304056,4.553680,5.105551,5.117330,5.119555


In [308]:
lr = LinearRegression()

### Years 1-3

In [309]:
# drop year 4 and year 5
x = base_data.drop(['Year 4 Sales - Inf', 'Year 5 Sales - Inf'], axis=1)
y = base_data['Year 5 Sales - Inf']

In [310]:
x.shape

(191, 3)

In [311]:
# fit model
lr.fit(x, y)

# coefficient of determination of the model
lr.score(x, y)

0.8181088870630957

In [278]:
#plt.figure(figsize=(15, 10), facecolor='white')

# residuals plot for the model
#sns.residplot(x=lr.predict(x), y='Year 5 Sales - Inf', data=base_data)

In [279]:
pred_1_3 = (pd.read_excel(file)
             .replace('no data', np.nan)
             [['Company', 'Name of NME Approved', 'Molecule ID', 
               'Year 1 Sales - Inf', 'Year 2 Sales - Inf', 
               'Year 3 Sales - Inf', 'Year 4 Sales - Inf', 
               'Year 5 Sales - Inf']]
            #drop rows that are missing values for year 1-3
             .dropna(axis=0, subset=['Year 1 Sales - Inf', 'Year 2 Sales - Inf', 'Year 3 Sales - Inf'])
             .reset_index()
             .drop('index', axis=1))

#only keep rows that are null in years 4 AND 5
pred_1_3 = pred_1_3[pred_1_3['Year 5 Sales - Inf'].isnull()]
pred_1_3 = pred_1_3[pred_1_3['Year 4 Sales - Inf'].isnull()]

In [280]:
x_1_3 = (pred_1_3.drop(['Company', 'Name of NME Approved', 'Molecule ID', 
               'Year 4 Sales - Inf', 'Year 5 Sales - Inf'], axis=1))

#add 1 and log transform
x_1_3 = (x_1_3 + 1).transform(lambda x : np.log(x))

In [281]:
# make sure you're fitting to the correct model here
pred_1_3['year 5 prediction'] = lr.predict(x_1_3)

# take inverse log
pred_1_3['year 5 prediction'] = [np.e**x for x in pred_1_3['year 5 prediction']]
pred_1_3['year 5 prediction'] = [x-1 for x in pred_1_3['year 5 prediction']]

### Years 2-4

In [312]:
# drop year 1 and year 5 (year 2-4)
x = base_data.drop(['Year 1 Sales - Inf', 'Year 5 Sales - Inf'], axis=1)
y = base_data['Year 5 Sales - Inf']

In [313]:
x.shape

(207, 3)

In [314]:
# build model
lr.fit(x, y)

# coefficient of determination of the model
lr.score(x, y)

0.9463770954009582

In [284]:
#plt.figure(figsize=(15, 10), facecolor='white')

# residual plot
#sns.residplot(x=lr.predict(x), y='Year 5 Sales - Inf', data=base_data)

In [285]:
pred_2_4 = (pd.read_excel(file)
             .replace('no data', np.nan)
             [['Company', 'Name of NME Approved', 'Molecule ID', 
               'Year 1 Sales - Inf', 'Year 2 Sales - Inf', 
               'Year 3 Sales - Inf', 'Year 4 Sales - Inf', 
               'Year 5 Sales - Inf']]
            # drop rows that are null in years 2-4
             .dropna(axis=0, subset=['Year 2 Sales - Inf', 'Year 3 Sales - Inf', 'Year 4 Sales - Inf'])
             .reset_index()
             .drop('index', axis=1))

# keep only what is null for year 5
pred_2_4 = pred_2_4[pred_2_4['Year 5 Sales - Inf'].isnull()]

In [286]:
x_2_4 = (pred_2_4.drop(['Company', 'Name of NME Approved', 'Molecule ID', 'Year 1 Sales - Inf',
                        'Year 5 Sales - Inf'], axis=1))

In [287]:
#added 2023
x_2_4 = (x_2_4 + 1).transform(lambda x : np.log(x))

In [288]:
# make sure you're fitting to the correct model here
pred_2_4['year 5 prediction'] = lr.predict(x_2_4)

# take inverse log
pred_2_4['year 5 prediction'] = [np.e**x for x in pred_2_4['year 5 prediction']]
pred_2_4['year 5 prediction'] = [x-1 for x in pred_2_4['year 5 prediction']]

In [291]:
# export to csv
pred_1_3.to_csv('year 1-3 predictions.csv', index=False)
pred_2_4.to_csv('year 2-4 predictions.csv', index=False)